In [6]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF

In [18]:
#Boilerplate spark stuff 
conf = SparkConf().setMaster("local").setAppName("sparkTFIDF")


In [9]:
rawData = sc.textFile("/Users/hamsinisankaran/Downloads/DataScience-Python3/subset-small.tsv")

In [10]:
#load documents one per line 
fields = rawData.map(lambda x: x.split("\t"))
documents = fields.map(lambda x: x[3].split(" "))

In [11]:
#store the documents name for later 
documentNames = fields.map(lambda x: x[1])

In [12]:
# Now hash the words in each document to their term frequencies:
hashingTF = HashingTF(100000)  #100K hash buckets just to save some memory
tf = hashingTF.transform(documents)

In [13]:
# At this point we have an RDD of sparse vectors representing each document,
# where each value maps to the term frequency of each unique hash value.

# Let's compute the TF*IDF of each term in each document:
tf.cache()
idf = IDF(minDocFreq=2).fit(tf)
tfidf = idf.transform(tf)

In [14]:
# Now we have an RDD of sparse vectors, where each value is the TFxIDF
# of each unique hash value for each document.

# I happen to know that the article for "Abraham Lincoln" is in our data
# set, so let's search for "Gettysburg" (Lincoln gave a famous speech there):

# First, let's figure out what hash value "Gettysburg" maps to by finding the
# index a sparse vector from HashingTF gives us back:
gettysburgTF = hashingTF.transform(["Gettysburg"])
gettysburgHashValue = int(gettysburgTF.indices[0])

In [15]:
# Now we will extract the TF*IDF score for Gettsyburg's hash value into
# a new RDD for each document:
gettysburgRelevance = tfidf.map(lambda x: x[gettysburgHashValue])

In [16]:
# We'll zip in the document names so we can see which is which:
zippedResults = gettysburgRelevance.zip(documentNames)

In [17]:
# And, print the document with the maximum TF*IDF value:
print("Best document for Gettysburg is:")
print(zippedResults.max())

Best document for Gettysburg is:
(33.13476250917198, 'Abraham Lincoln')
